# Open311 Classifier
Find within some exploration of the Open311DataSet and some classification experiments

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

# how about nltk though?

DATA_FOLDER = './data/'
reports = pd.read_csv(os.path.join(DATA_FOLDER, 'Open311Data.csv'))
reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92286 entries, 0 to 92285
Data columns (total 13 columns):
service_request_id    92286 non-null int64
requested_datetime    92286 non-null object
updated_datetime      92286 non-null object
closed_date           92286 non-null object
status                92286 non-null object
source                58419 non-null object
service_name          92286 non-null object
service_subtype       0 non-null float64
description           84009 non-null object
agency_responsible    24915 non-null object
address               86808 non-null object
lat                   74468 non-null float64
long                  74470 non-null float64
dtypes: float64(3), int64(1), object(9)
memory usage: 9.2+ MB


Of particular interest is the 'description' column. No doubt location, date, and time info could prove useful. First we just use text.

In [2]:
reports['service_name'].value_counts()

Trash                                 40124
Recycling                             18606
Excessive Growth                       7445
Yard Waste                             4586
Other                                  2556
Potholes, Other Street Repair          2020
Temporary Signage w/o permit           1730
Sewer Problems (Sanitary Sewers)       1467
Water Utility Problems                 1358
Sidewalk Snow Removal                  1297
Parking on Unimproved Surface          1009
Street Lights                           922
Graffiti                                749
Water Quality                           730
Traffic Related Complaints              654
Street Snow Removal                     635
Line of Sight                           572
Blocked Street                          558
Traffic Signals                         503
Website & Mobile Apps Feedback          451
Sidewalk & Curb Complaints              444
Parks & Playgrounds                     396
Unsafe Buildings                

In [3]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
nltk.download() # kicks off a GUI thing
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Oh snap. lots of NaN descriptions. That's no help!
Do your data massaging here

In [87]:
goodreports = reports[(pd.notnull(reports['description']))]

In [88]:
y = goodreports['service_name'] == 'Trash'
#y = goodreports['service_name'] == 'Recycling'
#y = goodreports['service_name'] == 'Excessive Growth'
#y = goodreports['service_name'] == 'Yard Waste'
#y = goodreports['service_name'] == 'Potholes, Other Street Repair'
#y = goodreports['service_name'] == 'Temporary Signage w/o permit'
#y = goodreports['service_name'] == 'Sewer Problems (Sanitary Sewers)'
#y = goodreports['service_name'] == 'Water Utility Problems'
#y = goodreports['service_name'] == 'Sidewalk Snow Removal'
#y = goodreports['service_name'] == 'Parking on Unimproved Surface'
#y = goodreports['service_name'] == 'Graffiti'
x = goodreports['description']

# do da splits 33 for test? 25 is better?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [89]:
# SVM time
# changing num_iter=5 to max_iter=5 made it go wild son
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, max_iter=5, random_state=42)),
                         ])

_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.8806766944414385

In [90]:
# Naive Bayes time
from sklearn.naive_bayes import MultinomialNB
text_clf_nb = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                         ])

_ = text_clf_nb.fit(X_train, y_train)
predicted_nb = text_clf_nb.predict(X_test)
np.mean(predicted_nb == y_test)

0.8717671247700466

It'd be a not too bad idea to look at where/how it failed

In [91]:
# wrong predictions...very few (sewer deal)
X_test[predicted_nb]
# it's what we seek
#X_test[predicted_nb == False]

20134                     Bags must have sticker attached.
24553                       10/24/07\r\ncan exceeds 35 gal
66568                                            too heavy
4941       Trash piled behind duplex and blowing in alley.
41568                             Missed pick-up/recycling
27321                          had tv in trash for pick up
34295    Missed pick-up/ This is the second week she ha...
22617                                          no stickers
37462                                 no stickers on trash
39480                                         S-no sticker
44155                    TYR##OF##\nREQUEST FOR FLAG STOP.
10802    Bags/cans must have sticker attached.  Boxes n...
37057                                           no sticker
36820                                           no sticker
46528    Resident has been written up many times in 201...
17060                                    Courtesy Pick up.
66979                                  ###B###\n2nd noti

In [93]:
text_clf_svm.predict(['sticker'])

array([ True])

In [94]:
y_test.value_counts()

False    15548
True     12175
Name: service_name, dtype: int64

In [95]:
#X_test[y_test]
#X_test[predicted_nb]

from sklearn.metrics import f1_score, precision_score, recall_score
fmeasure1 = f1_score(y_test, predicted_nb, average="macro")
fmeasure2 = f1_score(y_test, predicted_nb, average="micro")

precision = precision_score(y_test, predicted_nb, average="macro")
recall = recall_score(y_test, predicted_nb, average="macro")

fmeasure1,fmeasure2,precision,recall

(0.8683757524312836,
 0.8717671247700466,
 0.8745249960366767,
 0.8650427106835875)

Numbers below are Naive Bayes

So in the example of the sewer, it says '99%' correct prediction, but out of 476 it only picks up 247 (low sensitivity) - so it's sort of like the accuracy of a fraud detection alg. that always says 'there's no fraud'

For recycling:
flagged 3754 while there are 5131

Growth
947 flagged, 2190 in fact. Hmmm :(

Next things:
confusion matrices
'balance' the data?

For graffiti, precision, recall

0.9406737870149864,
 0.5184150018713973
 
NEXT:
Tree methods
Logistic Regression
Is label encoding legit?
What would the russian kids do?

The multiclass thing doesn't look good at all really

 

In [9]:
# ROC Curve
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_nb, pos_label=2)
metrics.auc(fpr, tpr)

/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/metrics/ranking.py:571: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


nan

In [115]:
from sklearn.preprocessing import label_binarize
lb = preprocessing.LabelBinarizer()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit([1, 2, 2, 6])
print(le.classes_)
print(le.transform([1, 1, 2, 6])) 
print(le.inverse_transform([0, 0, 1, 2]))
le.fit(['bobs','burgers','is','funny'])
print(le.classes_)
print(le.transform(['bobs', 'bobs', 'funny'])) 
print(le.inverse_transform([0, 0, 2]))

[1 2 6]
[0 0 1 2]
[1 1 2 6]
['bobs' 'burgers' 'funny' 'is']
[0 0 2]
['bobs' 'bobs' 'funny']


/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [133]:
#le.fit(goodreports['service_name'].unique())
#y_vect = le.transform(goodreports['service_name'])
#print(y_vect)
#print(np.max(y_vect))
# I do not believe this worked!
#y = label_binarize(y_vect, classes=le.classes_) # [0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 ,10])
# can just jump straight to binarizer, but....
yz = lb.fit_transform(goodreports['service_name'])
print(yz[84000])
#lb.fit_transform(['eat',' a shit','and','die'])
print(goodreports['service_name'][84000])
goodreports['service_name']

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Recycling


0             Excessive Growth
1             Excessive Growth
2             Excessive Growth
3             Excessive Growth
4             Excessive Growth
5             Excessive Growth
11            Excessive Growth
13            Excessive Growth
14            Excessive Growth
15            Excessive Growth
16            Excessive Growth
17            Excessive Growth
18            Excessive Growth
20            Excessive Growth
21            Excessive Growth
22            Excessive Growth
23            Excessive Growth
24            Excessive Growth
25            Excessive Growth
26            Excessive Growth
27            Excessive Growth
28            Excessive Growth
29            Excessive Growth
30            Excessive Growth
31            Excessive Growth
32            Excessive Growth
33            Excessive Growth
34            Excessive Growth
35            Excessive Growth
36            Excessive Growth
                 ...          
92256         Blocked sidewalk
92257   

In [134]:
# this one is highly dubious!
from sklearn import svm
print(len(y))
x=goodreports['description']
print(len(x))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
# Learn to predict each class against the other
from sklearn.multiclass import OneVsRestClassifier
text_clf_svc = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=7))),
                         ])

_ = text_clf_svc.fit(X_train, y_train)
predicted_svc = text_clf_svc.predict(X_test)
np.mean(predicted_svc == y_test)


84009
84009


/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))
/Users/scharlesworth/.pyenv/versions/3.6.3/envs/open-311-env/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/Users/scharlesworth

1.0

In [104]:

from sklearn.metrics import f1_score, precision_score, recall_score
fmeasure1 = f1_score(y_test, predicted_svc, average="macro")
fmeasure2 = f1_score(y_test, predicted_svc, average="micro")

precision = precision_score(y_test, predicted_svc, average="macro")
recall = recall_score(y_test, predicted_svc, average="macro")

fmeasure1,fmeasure2,precision,recall

(0.9009948386136113, 0.9030047253183278, 0.9038392505524209, 0.898976993284095)

In [105]:
predicted_svc

array([ True, False, False, ..., False,  True, False])

In [106]:
y_train

75850    False
28960     True
4906     False
75999    False
56174    False
30516     True
385      False
24331     True
9534     False
10430     True
14703     True
52986    False
75053    False
50268    False
60089    False
24406     True
20460     True
59632    False
7057     False
24608     True
44145     True
43586     True
9883     False
42826     True
30588     True
19602     True
62014    False
24077     True
16301     True
65148    False
         ...  
72764    False
36772     True
71053    False
63641    False
2793     False
19377     True
26613     True
77266    False
29669     True
58025    False
91374    False
5556     False
73298    False
70250    False
68248    False
64241    False
803      False
70145    False
72550    False
42099     True
16957     True
64773    False
46382     True
38194     True
90654    False
6893     False
59222    False
83317    False
894      False
16725     True
Name: service_name, Length: 56286, dtype: bool